In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch9.06_LangChain과vectorDatabase을 활용한 RAG구현</span>

# [RAG 절차]

1. 문서를 읽는다.
   %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다.
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장)  cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변을 얻음.(렝체인 사용 가능)
    %pip install -q langchain
    
    (https://smith.langchain.com/에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)
    





# 0. 패키지 설치


In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [3]:
# 텍스트를 chunk로 나누는 기능만 있는 경향 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [7]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 5 s
Wall time: 5.5 s


In [8]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters

## 2.1 1500토큰씩 쪼개서 읽어오기


In [9]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader('./data/소득세법(법률)(제21065호)(20260102).docx')
# gpt-4, gpt4o, gpt-4 turbo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", # 토큰을 세는 방식 이름
    chunk_size=1500,             # chunk당 토큰 수 기준
    chunk_overlap=200
    #separators = ["\n","\n\n"]파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() -start
print("문서를 쪼개면서 읽는 시간 : ", runtime)

문서를 쪼개면서 읽는 시간 :  6.944356203079224


In [11]:
len(documents)  # chunk 수

180

In [16]:
# chunk 글자수
# documents[0].page_content   # dir(documents)
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [21]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents[:-1]]))

2060
955
1287


In [23]:
print(documents[0].page_content)

소득세법

소득세법

[시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]

기획재정부(재산세제과(양도소득세)) 044-215-4312

기획재정부(소득세제과(근로소득)) 044-215-4216

기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233

기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217



제1장 총칙 <개정 2009. 12. 31.>



제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.

[본조신설 2009. 12. 31.]

[종전 제1조는 제2조로 이동 <2009. 12. 31.>]



제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>

1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.

2. “비거주자”란 거주자가 아닌 개인을 말한다.

3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.

4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.

5. “사업자”란 사업소득이 있는 거주자를 말한다.

② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.

[본조신설 2009. 12. 31.]



제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인

② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자

3. 내국법인

4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 

## 2.2 1500글자씩 쪼개서 읽어오기

In [1]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader("data/소득세법(법률)(제21065호)(20260102).docx")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 1500글자씩 chunking
    chunk_overlap=200,  
    # separators=["\n\n","\n"," ",""] 기본값
)
# 재귀적으로 다음 순서대로 시도 :
# 1. \n\n(문단구분) 
# 2. \n(줄바꿈)
# 3. " "(공백)
# 4. "" - 최후에는 글자 단위로 chunking
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time()-start
print("문서를 1500글자쯤으로 쪼개면서 읽는 시간 :",runtime)
print("chunk 갯수 :",len(documents))

문서를 1500글자쯤으로 쪼개면서 읽는 시간 : 4.771714210510254
chunk 갯수 : 193


In [8]:
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents][:-1]))

1496
325
873


# 3. 쪼갠 문서를 임베딩 - > 벡터데이터베이스 저장
- 임베딩모델 : text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터데이터베이스(벡터 store) : chroma

In [11]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [28]:
# embed_query 한 문자열을 임베딩 벡터로 전환한 숫자list를 return
len(embedding.embed_query("소득세은 어쩌구"))

3072

In [12]:
embedding_vectors = embedding.embed_documents( # 여러 문자열을 임베딩 벡터로 
    [
       # "소득세법은 어쩌구",
        documents[0].page_content,
        documents[1].page_content
    ]
)

In [36]:
print(len(embedding_vectors),len(embedding_vectors[0]),len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

2 3072 3072
[0.01570405438542366, -0.01078017894178629, -0.0006093500414863229, 0.0261734239757061, 0.026762327179312706, 0.009318829514086246, -0.007159522268921137, 0.04168117791414261, -0.010114938020706177, 0.008413665927946568]


In [13]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
database = Chroma.from_documents(
    documents=documents,  # chunk
    embedding=embedding,  # 임베딩 객체
    collection_name="tax-collection",  # 생략시 이름 랜덤
    persist_directory="./chroma"       # 생략시 로컬DB에 저장 안됨 => 프로그램 종료시  DB 날라감
)

# 이미 저장된 vector DB(store)를 사용할 때
database = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma"
)

CPU times: total: 1.03 s
Wall time: 7.08 s


In [14]:
results = database._collection.get(include=['embeddings','documents','metadatas'])
print("데이터 수 : ",len(results['ids']))
print("문서 임베딩 차원 수 :",len(results['embeddings'][0]))
print("1번째 임베딩 샘플 :",results['embeddings'][1])
print("1번째 원본 :",results['documents'][1][:50])
print("1번째 metadata :",results['metadatas'][1])

데이터 수 :  193
문서 임베딩 차원 수 : 3072
1번째 임베딩 샘플 : [ 0.01674929 -0.01080192 -0.0115323  ...  0.00672168 -0.01151033
 -0.00994524]
1번째 원본 : 1. 구성원 간 이익의 분배비율이 정하여져 있고 해당 구성원별로 이익의 분배비율이 확인되는
1번째 metadata : {'source': 'data/소득세법(법률)(제21065호)(20260102).docx'}


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [48]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query=query,
                                           k=2) # 기본k 값은 4

In [49]:
#retrieved_docs

[Document(id='0431448c-8c8c-4b43-a180-02e5494eb46d', metadata={'source': './data/소득세법(법률)(제21065호)(20260102).docx'}, page_content='8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1., 2025. 12. 23.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 3천만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는

In [53]:
#print("\n\n---\n\n".join([doc.page_content for doc in retrieved_docs]))
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 5. 유사도 검색으로 가져온 문서(retrieved_doc)를 질문과 같이 LLM에 전달하여 답변 생성 -1

In [55]:
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-4.1-nano")

In [56]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_doc}
질문:{query}"""

In [57]:
ai_message = llm.invoke(prompt)

In [62]:
ai_message.usage_metadata

{'input_tokens': 2132,
 'output_tokens': 828,
 'total_tokens': 2960,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [64]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 기본적으로 근로소득에 대한 과세표준과 세율, 그리고 공제액 등을 고려하여 진행됩니다. 아래는 대략적인 계산 과정입니다.

1. 근로소득 공제액 산정  
- 근로소득에 대한 공제는 근로소득자가 일정 금액까지 면제받는 제도입니다.  
- 2023년 기준으로 근로소득 공제는 다음과 같습니다.  
  - 연봉 5천만원인 경우:  
    근로소득 공제 = 연봉 × 25% + 1,200,000원 (최소 공제액이 있기 때문에, 필요시 최댓값을 적용합니다.  
    하지만, 일반적으로 연봉이 4,600만원 이상이면 공제액은 연봉의 15% 이상이 되지 않도록 제한됩니다.

2. 과세표준 계산  
- 연봉(5,000만원)에서 근로소득 공제액을 빼면 과세표준이 나옵니다.

3. 세율 적용  
- 과세표준에 따른 소득세율을 적용합니다.

※ 참고로, 2023년 기준 소득세율은 다음과 같습니다.

| 과세표준(원) | 세율(%) | 누진공제(원) |
|--------------|--------|--------------|
| 1,200만원 이하 | 6%     | 0           |
| 1,200만원 초과 ~ 4,600만원 이하 | 15%    | 108만원     |
| 4,600만원 초과 ~ 8,800만원 이하 | 24%    | 522만원     |
| 8,800만원 초과 ~ 1억5천만원 이하 | 35%    | 1,490만원   |
| 1억5천만원 초과 ~ 3억원 이하 | 38%    | 1,950만원   |
| 3억원 초과 | 40%    | 3,300만원   |

4. 대략적인 계산 예시

- 연봉: 50,000,000원

- 근로소득 공제 (2023년 적용 예시):  
  공제액 = 연봉 × 25% + 1,200,000원 = 12,500,000 + 1,200,000 = 13,700,000원 (최대 공제액 제한 고려 필요)

- 과세표준: 50,000,000원 - 13,700,000원 = 36,300,000원

- 소득세 계

# 5. 유사도 검색으로 가져온 문서(retrieved_doc)를 질문과 같이 LLM에 전달하여 답변 생성 -2

In [65]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
promptTemplate = ChatPromptTemplate([
    ("system","당신은 최고의 한국 소득세 전문가입니다"),
    ("human",f"""다음 문맥을 참고하여 질문에 답변하세요.
    답을 모르면 모른다고 말하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문:{{question}}
    문맥:{{context}}
    답변:""")
])
promptTemplate

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최고의 한국 소득세 전문가입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 문맥을 참고하여 질문에 답변하세요.\n    답을 모르면 모른다고 말하세요.\n    최대 3문장으로 간결하게 답변하세요.\n    질문:{question}\n    문맥:{context}\n    답변:'), additional_kwargs={})])

In [66]:
prompt = promptTemplate.invoke({'context':retrieved_doc, # retrieved_docs보다 추천
                                'question':query
                               })

ChatPromptValue(messages=[SystemMessage(content='당신은 최고의 한국 소득세 전문가입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='다음 문맥을 참고하여 질문에 답변하세요.\n    답을 모르면 모른다고 말하세요.\n    최대 3문장으로 간결하게 답변하세요.\n    질문:연봉 5천만원인 직장인의 소득세는 얼마인가요?\n    문맥:8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1., 2025. 12. 23.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 3천만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 

In [67]:
llm.invoke(prompt)

AIMessage(content='질문하신 연봉 5천만원인 직장인의 소득세는 구체적인 세액을 계산하려면 공제액과 세율, 기타 소득 및 세액공제 항목이 필요합니다. 따라서, 제공된 정보만으로 정확한 소득세 금액을 산출하기 어렵습니다. 일반적으로 근로소득세는 누진세율이 적용되며, 소득 전체 금액과 공제 내용을 고려해야 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 2153, 'total_tokens': 2252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-CvasJbtLakAz8ATHbtwzoP6ZGrj3y', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9b9f-77cd-78e3-b46f-75d149fa7525-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2153, 'output_tokens': 99, 'total_tokens': 2252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [69]:
# 위의 예제를 Langchain으로 답변 생성
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(promptTemplate.invoke({'context':retrieved_doc,
                                                       'question':query})))

'연봉 5천만원인 직장인의 경우, 소득세는 약 650만 원 내외입니다. 정확한 금액은 초과 누진세율 적용 및 공제사항에 따라 달라질 수 있습니다. 자세한 계산은 세무 전문가와 상담하시는 것이 좋습니다.'

# 6. langchain 으로 답변 생성
##   <span style="color:red">위의 예제를 langchain으로 답변 생성</span>

In [70]:
rag_chain = promptTemplate | llm | output_parser
rag_chain.invoke({'context':retrieved_doc,'question':query})

'연봉 5천만원인 직장인의 소득세는 구체적인 공제와 세율 계산이 필요하나, 대체로 약 700만 원 내외의 세액이 예상됩니다. 정확한 금액은 근로소득공제, 인적공제, 세액공제 등을 반영한 계산이 필요합니다. 따라서, 구체적인 세액 산출을 위해 상세한 소득 및 공제 내역이 요구됩니다.'

## langchain 전달
```
   * query -> retrieve 전달 -> prompt에 context에 삽입
   * smith.langchain.com에서 key생성 후 .env에 LANGCHAIN_API_KEY 추가
```

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model="gpt-4.1-mini")
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
# 2. vector store load
vectorstore = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma"
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":4})
# vectorstore.similarity_search("질문",k=2)  질문을 모르기때문에 사용 불가

# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장을 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 :"""
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return "\n\n---\n\n".join([doc.page_content for doc in documents])

In [2]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough()  # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    # llm에 prompt invoke
    | StrOutputParser()
)
# 7. 실행
query = " 연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득 공제 후 과세표준에 세율을 적용하여 계산합니다. 2024년 기준 간이세액표 상 대략 250만~300만원 내외로 예상되며, 정확한 세액은 공제항목과 세액공제에 따라 다릅니다. 상세 계산은 국세청 간이세액표 또는 홈택스 계산기를 참고하시기 바랍니다.'